In [2]:
import pandas as pd
import networkx as nx

In [ ]:
df = pd.read_csv("D:/Datasets/stream_tweets/df_finale_sardine.csv")

In [ ]:
df_relations = df[["screen_name","@mentions","hashtags"]]

In [ ]:
df_relations["hashtags"] = df_relations["hashtags"].replace("#","", regex = True).str.lower()

In [ ]:
df_relations_mentions = df_relations[(df_relations["@mentions"] != "Vuoto") & (df_relations["@mentions"].notnull())]

In [ ]:
df_relations_hashtags = df_relations[(df_relations["hashtags"] != "Vuoto") & (df_relations["hashtags"].notnull())]

In [ ]:
df_relations_mentions["splitted_mentions"] = df_relations_mentions["@mentions"].apply(lambda x: x.split(','))

In [ ]:
df_relations_mentions

In [ ]:
df_relations_hashtags["hashtags"] = df_relations_hashtags["hashtags"].replace("\[","", regex = True).str.lower()
df_relations_hashtags["hashtags"] = df_relations_hashtags["hashtags"].replace("\]","", regex = True).str.lower()
df_relations_hashtags["hashtags"] = df_relations_hashtags["hashtags"].replace("\'","", regex = True).str.lower()
df_relations_hashtags["hashtags"] = df_relations_hashtags["hashtags"].replace(" ","", regex = True).str.lower()

In [ ]:
df_relations_hashtags["splitted_hashtags"] = df_relations_hashtags["hashtags"].apply(lambda x: x.split(','))

In [ ]:
def from_list_to_nan(x):
    if x == ['']:
        x = None
        return x
    else:
        return x

In [ ]:
df_relations_hashtags["splitted_hashtags"] = df_relations_hashtags["splitted_hashtags"].apply(from_list_to_nan)

In [ ]:
df_relations_hashtags["splitted_hashtags"].head(120)

In [ ]:
df_relations_mentions["splitted_mentions"].head(20)

In [ ]:
def remove_comma(x):
    filtered = [i for i in x if i.strip()]
    return filtered

In [ ]:
df_relations_mentions["splitted_mentions"] = df_relations_mentions["splitted_mentions"].apply(remove_comma)

In [ ]:
df_relations_mentions.splitted_mentions

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
for r in df_relations_mentions.iterrows():
    for user in r[1]['splitted_mentions']:
        G.add_edge(r[1]['screen_name'], user)

In [ ]:
# Find modularity
import matplotlib.pyplot as plt
import community

part = community.best_partition(G)
mod = community.modularity(part,G)

# Plot, color nodes using community structure
values = [part.get(node) for node in G.nodes()] #cluster di appartenenza 
plt.figure(figsize=(60, 60))
nx.draw_spring(G, cmap=plt.get_cmap('jet'), node_color = values, node_size=40, with_labels=False)
plt.show()

In [ ]:
[part.get(node) for node in G.nodes() if ]

In [ ]:
cluster[400]

In [ ]:
grado = pd.DataFrame(nx.degree(G))

In [ ]:
grado.sort_values(1, ascending=False).head(20)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(80, 80))
nx.draw(G, with_labels=False)
plt.title('Rappresentazione a grafo dei tweets associati alle sardine', size=5)
plt.show()

In [ ]:
# Find modularity
import matplotlib.pyplot as plt
import community

part = community.best_partition(G)
mod = community.modularity(part,G)

# Plot, color nodes using community structure
values = [part.get(node) for node in G.nodes()] #cluster di appartenenza 
plt.figure(figsize=(60, 60))
nx.draw_spring(G, cmap=plt.get_cmap('jet'), node_color = values, node_size=40, with_labels=False)
plt.show()

In [ ]:
values

## Hashtag relations

In [ ]:
hashtag_relations = df_relations[df_relations["splitted_hashtag"].notnull()]

In [ ]:
G2 = nx.Graph()
for r in hashtag_relations.iterrows():
    for user in r[1]['splitted_hashtag']:
        G2.add_edge(r[1]['screen_name'], user)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(80, 80))
nx.draw(G2, with_labels=False)
plt.title('Rappresentazione a grafo dei tweets associati agli hashtag usati', size=5)
plt.show()

In [ ]:
from pyvis.network import Network
g = Network()

In [ ]:
g.toggle_hide_edges_on_drag(False)
#g.show_buttons()
g.barnes_hut()
g.from_nx(G)
g.set_options("""
var options = {
  "nodes": {
    "color": {
      "border": "rgba(127,60,233,1)",
      "background": "rgba(148,227,252,1)"
    },
    
    "font": {
      "size": 87,
      "face": "tahoma",
      "strokeWidth": 22
    },
    "hidden": true
  },
  "edges": {
    "color": {
      "highlight": "rgba(132,132,132,1)",
      "inherit": false
    },
    "dashes": true,
    "smooth": false
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -80000,
      "springLength": 250,
      "springConstant": 0.001
    },
    "minVelocity": 0.75
  }
}
""")
g.show("ex.html")


# Cluster escludendo nodi sconnessi per mentions: 

In [ ]:
df_for_men = pd.read_csv("D:/Datasets/stream_tweets/cluster_mention.csv")

In [ ]:
df_for_men["@mentions"] = df_for_men["@mentions"].replace(" ","", regex = True).str.lower()
df_for_men["@mentions"] = df_for_men["@mentions"].replace("\[","", regex = True).str.lower()
df_for_men["@mentions"] = df_for_men["@mentions"].replace("\]","", regex = True).str.lower()
df_for_men["@mentions"] = df_for_men["@mentions"].replace("\'","", regex = True).str.lower()
#df_for_men["@mentions"] = df_for_men["@mentions"].apply(lambda x: x.split(','))

In [ ]:
num_clust = df_for_men["cluster"].value_counts().head(21).reset_index()["index"]

In [ ]:
new_df = df_for_men[df_for_men["cluster"].isin(num_clust)]

In [ ]:
app = new_df[["@mentions","screen_name"]]

In [ ]:
app2 = app["@mentions"].str.split(",", expand = True)

In [ ]:
app2["name"] = app["screen_name"]
app3=app2.melt(id_vars = "name")
app3.drop(["variable"],axis=1,inplace=True)
app3.dropna(inplace=True)
app3.rename(columns={"name": "from", "value": "to"},inplace=True)

In [ ]:
app3["to"] = app3["to"].apply(lambda x: x.split(','))

In [ ]:
G2 = nx.Graph()
for r in app3.iterrows():
    for user in r[1]['to']:
        G2.add_edge(r[1]['from'], user)

In [ ]:
# Find modularity
import matplotlib.pyplot as plt
import community

part = community.best_partition(G2)
mod = community.modularity(part,G2)

# Plot, color nodes using community structure

"""
values = [part.get(node) for node in G2.nodes()] #cluster di appartenenza 
plt.figure(figsize=(60, 60))
nx.draw_spring(G2, cmap=plt.get_cmap('jet'), node_color = values, node_size=40, with_labels=False)
plt.savefig('D:/Datasets/stream_tweets/cluster_mentions.png')
plt.show()
"""



In [ ]:
nome = pd.DataFrame.from_dict(part, orient = "index").reset_index()

In [ ]:
nome.rename(columns = {"index":"name", 0:"cluster"}, inplace = True)

In [ ]:
connessioni = pd.DataFrame(G2.edges, columns = ["nome","tag"])

In [ ]:
ment_clu = pd.merge(connessioni, nome, left_on = "nome", right_on = "name")[["nome","tag","cluster"]]

In [ ]:
ment_clu  = pd.merge(ment_clu, nome, left_on = "tag", right_on = "name").drop("name", axis = 1)

In [ ]:
ment_clu = ment_clu[ment_clu["cluster_x"] == ment_clu["cluster_y"]].drop("cluster_x", axis = 1)\
            .rename(columns = {"cluster_y":"cluster"})

In [ ]:
ment_clu1 = ment_clu[ment_clu["cluster"] == 0]
ment_clu2 = ment_clu[ment_clu["cluster"] == 1]
ment_clu3 = ment_clu[ment_clu["cluster"] == 3]
ment_clu4 = ment_clu[ment_clu["cluster"] == 6]
ment_clu5 = ment_clu[ment_clu["cluster"] == 9]

In [ ]:
ment_clu6 = ment_clu[ment_clu["cluster"] == 2]

In [ ]:
#ment_clu1.to_csv("D:/Datasets/stream_tweets/ment_clu1.csv")
#ment_clu2.to_csv("D:/Datasets/stream_tweets/ment_clu2.csv")
#ment_clu3.to_csv("D:/Datasets/stream_tweets/ment_clu3.csv")
#ment_clu4.to_csv("D:/Datasets/stream_tweets/ment_clu4.csv")
#ment_clu5.to_csv("D:/Datasets/stream_tweets/ment_clu5.csv")
#ment_clu6.to_csv("D:/Datasets/stream_tweets/ment_clu6.csv")

In [ ]:
ment_clu[ment_clu["nome"] == "matteosalvinimi"]

In [ ]:
#esportiamo il nuovo df finale con tutte le informazioni
#nome.rename(columns={"index": "name",0:"cluster"},inplace=True)
#finale_cluster_m=pd.merge(new_df, nome, left_on='screen_name', right_on='name')

In [ ]:
#finale_cluster_m.to_csv("D:/Datasets/stream_tweets/df_finale_true_cluster.csv")

In [18]:
ment_clu1 = pd.read_csv("D:/Datasets/stream_tweets/ment_clu3.csv")

In [19]:
ment_clu1

,Unnamed: 0,nome,tag,cluster
0,3410,dukana2,Jambonet1,3
1,3749,fattoquotidiano,1guadagn,3
2,3887,dukana2,CormaGodra76,3
3,3890,virginiaraggi,CormaGodra76,3
4,3963,dukana2,Ori254,3
...,...,...,...,...
1385,40295,JUREGRANDE,raisabato24,3
1386,40298,LoPan84773484,anselmidanilo,3
1387,40387,GliStessiChe,manuelparente,3
1388,40392,margheritamart4,elena07161968,3


In [16]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

def make_graph(ment_clu1):
    ment_clu1.drop("cluster", axis = 1, inplace = True)
    A = list(ment_clu1["nome"].unique())
    B = list(ment_clu1["tag"].unique())
    node_list = list(set(A+B))
    G = nx.Graph()
    for i in node_list:
        G.add_node(i)
    for i,j in ment_clu1.iterrows():
        G.add_edges_from([(j["nome"],j["tag"])])
    pos = nx.spring_layout(G, k=0.5, iterations=50)
    for n, p in pos.items():
        G.nodes[n]['pos'] = p
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'))
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        #hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Blackbody',
            reversescale=True,
            color=[],
            size=15,
            colorbar=dict(
                thickness=10,
                title='Numero di connessioni',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=0)))
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color']+=tuple([len(adjacencies[1])])
        node_info = str(adjacencies[0]) +' # di connessioni: '+str(len(adjacencies[1]))
        node_trace['text']+=tuple([node_info])

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='Cluster 1 mentions',
                    titlefont=dict(size=20),
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),

                        #annotations=[ dict(
                       #text="",
                       # showarrow=False,
                      #  xref="paper", yref="paper") ],


                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    iplot(fig)
    return fig

In [20]:
make_graph(ment_clu1)

Figure({
    'data': [{'line': {'color': '#888', 'width': 0.5},
              'type': 'scatter',
              'uid': 'e6603d90-3200-11ea-9135-08719087ac1b',
              'x': [-0.016536829993128777, -0.07874705642461777, None, ...,
                    -0.04290737956762314, -0.04290737956762314, None],
              'y': [0.7397098541259766, -0.025287389755249023, None, ...,
                    0.015660645440220833, 0.015660645440220833, None]},
             {'marker': {'color': [3, 5, 3, ..., 1, 2, 58],
                         'colorbar': {'thickness': 10,
                                      'title': 'Numero di connessioni',
                                      'titleside': 'right',
                                      'xanchor': 'left'},
                         'colorscale': 'Blackbody',
                         'line': {'width': 0},
                         'reversescale': True,
                         'showscale': True,
                         'size': 15},
              'm

In [6]:
ment_clu1.drop("cluster", axis = 1, inplace = True)

In [7]:
A = list(ment_clu1["nome"].unique())
B = list(ment_clu1["tag"].unique())
node_list = list(set(A+B))
G = nx.Graph()
for i in node_list:
    G.add_node(i)

In [8]:
for i,j in ment_clu1.iterrows():
    G.add_edges_from([(j["nome"],j["tag"])])

In [9]:
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.nodes[n]['pos'] = p

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
# installare libreria plotly versione 3.1.0

In [ ]:
 ['Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu',
            'Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet',
            'Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis']

In [ ]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'))
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Blackbody',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Numero di connessioni',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [ ]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = str(adjacencies[0]) +' # di connessioni: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Cluster 1 mentions',
                titlefont=dict(size=20),
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
             
                    #annotations=[ dict(
                   #text="",
                   # showarrow=False,
                  #  xref="paper", yref="paper") ],
             
             
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
iplot(fig)

# Cluster escludendo nodi sconnessi per hashtags

In [ ]:
df_for_hash = pd.read_csv("D:/Datasets/stream_tweets/cluster_hashtag.csv")

In [ ]:
df_for_hash["hashtags"] = df_for_hash["hashtags"].replace(" ","", regex = True).str.lower()
df_for_hash["hashtags"] = df_for_hash["hashtags"].replace("\[","", regex = True).str.lower()
df_for_hash["hashtags"] = df_for_hash["hashtags"].replace("\]","", regex = True).str.lower()
df_for_hash["hashtags"] = df_for_hash["hashtags"].replace("\'","", regex = True).str.lower()
#df_for_hash["hashtags"] = df_for_hash["hashtags"].apply(lambda x: x.split(','))

In [ ]:
num_clust = df_for_hash["cluster"].value_counts().head(17).reset_index()["index"]

In [ ]:
new_df = df_for_hash[df_for_hash["cluster"].isin(num_clust)]

In [ ]:
app = new_df[["hashtags","screen_name"]]
app2 = app["hashtags"].str.split(",", expand = True)
app2["name"] = app["screen_name"]
app3=app2.melt(id_vars = "name")
app3.drop(["variable"],axis=1,inplace=True)
app3.dropna(inplace=True)
app3.rename(columns={"name": "from", "value": "to"},inplace=True)

In [ ]:
app3["to"] = app3["to"].apply(lambda x: x.split(','))

In [ ]:
G2 = nx.Graph()
lista = []
lista2 = []
for r in app3.iterrows():
    for user in r[1]['to']:
        G2.add_edge(r[1]['from'], user)

In [ ]:
# Find modularity
import matplotlib.pyplot as plt
import community

part = community.best_partition(G2)
mod = community.modularity(part,G2)

# # Plot, color nodes using community structure
# values = [part.get(node) for node in G2.nodes()] #cluster di appartenenza 
# plt.figure(figsize=(60, 60))
# nx.draw_spring(G2, cmap=plt.get_cmap('jet'), node_color = values, node_size=40, with_labels=False)
# plt.savefig('D:/Datasets/stream_tweets/cluster_hashtags.png')
# plt.show()

In [ ]:
nome = pd.DataFrame.from_dict(part, orient = "index").reset_index()

In [ ]:
nome.rename(columns={"index": "name",0:"cluster"},inplace=True)
finale_cluster_m=pd.merge(new_df, nome, left_on='screen_name', right_on='name')
#finale_cluster_m.to_csv("D:/Datasets/stream_tweets/df_finale_true_cluster_hashtags.csv")

In [ ]:
connessioni = pd.DataFrame(G2.edges, columns = ["nome","tag"])

In [ ]:
ment_clu = pd.merge(connessioni, nome, left_on = "nome", right_on = "name")[["nome","tag","cluster"]]
ment_clu  = pd.merge(ment_clu, nome, left_on = "tag", right_on = "name").drop("name", axis = 1)
ment_clu = ment_clu[ment_clu["cluster_x"] == ment_clu["cluster_y"]].drop("cluster_x", axis = 1)\
            .rename(columns = {"cluster_y":"cluster"})

In [ ]:
ment_clu

In [ ]:
hast_clu1 =ment_clu[ment_clu["cluster"] == 0]
hast_clu2 =ment_clu[ment_clu["cluster"] == 10]
hast_clu3 =ment_clu[ment_clu["cluster"] == 1]
hast_clu4 =ment_clu[ment_clu["cluster"] == 3]
hast_clu5 =ment_clu[ment_clu["cluster"] == 11]
hast_clu6 =ment_clu[ment_clu["cluster"] == 2]

In [ ]:
make_graph(hast_clu2)

In [ ]:
# hast_clu1.to_csv("D:/Datasets/stream_tweets/hast_clu1.csv")
# hast_clu2.to_csv("D:/Datasets/stream_tweets/hast_clu2.csv")
# hast_clu3.to_csv("D:/Datasets/stream_tweets/hast_clu3.csv")
# hast_clu4.to_csv("D:/Datasets/stream_tweets/hast_clu4.csv")
# hast_clu5.to_csv("D:/Datasets/stream_tweets/hast_clu5.csv")
# hast_clu6.to_csv("D:/Datasets/stream_tweets/hast_clu6.csv")